In [1]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
from html_table_parser import parser_functions as parser
import numpy as np

#### 등록된 api로 접속

In [2]:
API_KEY='*****************************' #authority key
company_code='005930' #company code
start_date='20170101'

#### 사업 보고서 종류 확인

In [3]:
url ='https://opendart.fss.or.kr/api/list.xml?crtfc_key='+API_KEY+'&corp_code='+company_code+'&bgn_de='+start_date+'&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003'
resultXML=urlopen(url)
result=resultXML.read()

In [4]:
xmlsoup=BeautifulSoup(result,'html.parser')

In [5]:
data = pd.DataFrame()
for t in xmlsoup.findAll('list'):
    temp=pd.DataFrame(([[t.corp_code.string,t.corp_name.string,t.stock_code.string,t.report_nm.string,
         t.rcept_no.string,t.flr_nm.string]]),
         columns=["corp_code","corp_name","stock_code","report_nm","rcept_no","flr_nm"])
    data=pd.concat([data,temp])

data=data.reset_index(drop=True)

In [6]:
data.head()

,corp_code,corp_name,stock_code,report_nm,rcept_no,flr_nm
0,00126380,삼성전자,005930,분기보고서 (2019.09),20191114001273,삼성전자
1,00126380,삼성전자,005930,반기보고서 (2019.06),20190814002218,삼성전자
2,00126380,삼성전자,005930,분기보고서 (2019.03),20190515001605,삼성전자
3,00126380,삼성전자,005930,사업보고서 (2018.12),20190401004781,삼성전자
4,00126380,삼성전자,005930,분기보고서 (2018.09),20181114001530,삼성전자


#### 각각의 url을 활용하여 재무제표 뽑기

In [7]:
a2019_09="http://dart.fss.or.kr/report/viewer.do?rcpNo=20191114001273&dcmNo=6958001&eleId=15&offset=1099327&length=69959&dtd=dart3.xsd"
a2019_06='http://dart.fss.or.kr/report/viewer.do?rcpNo=20190814002218&dcmNo=6846651&eleId=15&offset=1080041&length=69283&dtd=dart3.xsd'
a2019_03='http://dart.fss.or.kr/report/viewer.do?rcpNo=20190515001605&dcmNo=6738798&eleId=15&offset=1026852&length=68820&dtd=dart3.xsd'
a2018_12='http://dart.fss.or.kr/report/viewer.do?rcpNo=20190401004781&dcmNo=6616741&eleId=15&offset=1332659&length=90711&dtd=dart3.xsd'
a2018_09='http://dart.fss.or.kr/report/viewer.do?rcpNo=20181114001530&dcmNo=6382016&eleId=15&offset=1108496&length=72539&dtd=dart3.xsd'
a2018_06='http://dart.fss.or.kr/report/viewer.do?rcpNo=20180814001113&dcmNo=6282263&eleId=15&offset=1102087&length=72265&dtd=dart3.xsd'
a2018_03='http://dart.fss.or.kr/report/viewer.do?rcpNo=20180515001699&dcmNo=6180129&eleId=15&offset=1047985&length=72075&dtd=dart3.xsd'
a2017_12='http://dart.fss.or.kr/report/viewer.do?rcpNo=20180402005019&dcmNo=6060273&eleId=15&offset=1350131&length=91652&dtd=dart3.xsd'
a2017_09='http://dart.fss.or.kr/report/viewer.do?rcpNo=20171114001694&dcmNo=5845320&eleId=15&offset=1135961&length=73638&dtd=dart3.xsd'
a2017_06='http://dart.fss.or.kr/report/viewer.do?rcpNo=20170814001832&dcmNo=5752868&eleId=15&offset=1111624&length=73272&dtd=dart3.xsd'
a2017_03='http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=15&offset=1101560&length=112166&dtd=dart3.xsd'

In [8]:
url_list=[a2017_03, a2017_06, a2017_09, a2017_12, a2018_03, a2018_06, a2018_09, a2018_12, a2019_03, a2019_06, a2019_09]

#### 재무상태표 칼럼

In [9]:
a2019_09_col=['구분','제 51 기 3분기말','제 50 기말']
a2019_06_col=['구분','제 51 기 반기말','제 50 기말']
a2019_03_col=['구분','제 51 기 1분기말','제 50 기말']
a2018_12_col=['구분','제 50 기','제 49 기','제 48 기']
a2018_09_col=['구분','제 50 기 3분기말','제 49 기말']
a2018_06_col=['구분','제 50 기 반기말','제 49 기말']
a2018_03_col=['구분','제 50 기 1분기말','제 49 기말']
a2017_12_col=['구분','제 49 기','제 48 기','제 47 기']
a2017_09_col=['구분','제 49 기 3분기말','제 48 기말']
a2017_06_col=['구분','제 49 기 반기말','제 48 기말']
a2017_03_col=['구분','제 49 기 1분기말','제 48 기말','제 47 기말']

columns_a=[a2017_03_col,a2017_06_col,a2017_09_col,a2017_12_col,a2018_03_col,a2018_06_col,a2018_09_col,a2018_12_col
         ,a2019_03_col,a2019_06_col,a2019_09_col]

#### 손익계산서 칼럼

In [10]:
b2017_03_col=['구분','제 49 기 1분기3개월','제 49 기 1분기누적','제 48 기 1분기3개월','제 48 기 1분기누적','제 48 기','제 47 기']
b2017_06_col=['구분','제 49 기 반기3개월','제 49 기 반기누적','제 48 기 반기3개월','제 48 기 반기누적']
b2017_09_col=['구분','제 49 기 3분기3개월', '제 49 기 3분기누적', '제 48 기 3분기3개월', '제 48 기 3분기누적']
b2017_12_col=['구분','제 49 기','제 48 기','제 47 기']
b2018_03_col=['구분','제 50 기 1분기3개월', '제 50 기 1분기누적', '제 49 기 1분기3개월', '제 49 기 1분기누적']
b2018_06_col=['구분','제 50 기 반기3개월', '제 50 기 반기누적', '제 49 기 반기3개월', '제 49 기 반기누적']
b2018_09_col=['구분','제 50 기 3분기3개월','제 50 기 3분기누적','제 49 기 3분기3개월','제 49 기 3분기누적']
b2018_12_col=['구분','제 50 기','제 49 기','제 48 기']
b2019_03_col=['구분','제 51 기 1분기3개월', '제 51 기 1분기누적', '제 50 기 1분기3개월', '제 50 기 1분기누적']
b2019_06_col=['구분','제 51 기 반기3개월', '제 51 기 반기누적' ,'제 50 기 반기3개월', '제 50 기 반기누적']
b2019_09_col=['구분','제 51 기 3분기3개월', '제 51 기 3분기누적', '제 50 기 3분기3개월', '제 50 기 3분기누적']

columns_b=[b2017_03_col,b2017_06_col,b2017_09_col,b2017_12_col,b2018_03_col,b2018_06_col,b2018_09_col,b2018_12_col
         ,b2019_03_col,b2019_06_col,b2019_09_col]

#### 현금흐름표 칼럼

In [11]:
c2019_09_col=['구분','제 51 기 3분기','제 50 기 3분기']
c2019_06_col=['구분','제 51 기 반기','제 50 기 반기']
c2019_03_col=['구분','제 51 기 1분기','제 50 기 1분기']
c2018_12_col=['구분','제 50 기','제 49 기','제 48 기']
c2018_09_col=['구분','제 50 기 3분기','제 49 기 3분기']
c2018_06_col=['구분','제 50 기 반기','제 49 기 반기']
c2018_03_col=['구분','제 50 기 1분기','제 49 기 1분기']
c2017_12_col=['구분','제 49 기','제 48 기','제 47 기']
c2017_09_col=['구분','제 49 기 3분기','제 48 기 3분기']
c2017_06_col=['구분','제 49 기 반기','제 48 기 반기']
c2017_03_col=['구분','제 49 기 1분기','제 48 기 1분기','제 48 기','제 47 기']

columns_c=[c2017_03_col,c2017_06_col,c2017_09_col,c2017_12_col,c2018_03_col,c2018_06_col,c2018_09_col,c2018_12_col
         ,c2019_03_col,c2019_06_col,c2019_09_col]

## dataframe 정리

#### 재무상태표

In [12]:
url_list_1=[url_list[0],url_list[3]]
columns_a_1=[columns_a[0],columns_a[3]]


data1 = pd.DataFrame()

for i,j in zip(url_list_1, columns_a_1):
    
    report=urlopen(i)
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")
    
    # 재무상태표
    p1 = parser.make2d(table[1],) 
    BS = pd.DataFrame(p1[1:],columns=j)
    BS=BS.loc[[1,12,21,23,34,39,48,49],:]
    data1=pd.concat([data1,BS],axis=1)
data1=data1.reset_index()

In [13]:
url_list_2=[url_list[1],url_list[2]]
columns_a_2=[columns_a[1],columns_a[2]]

data2 = pd.DataFrame()

for i,j in zip(url_list_2, columns_a_2):
    
    report=urlopen(i)
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")
    
    # 재무상태표
    p1 = parser.make2d(table[1],) 
    BS = pd.DataFrame(p1[1:],columns=j)
    BS=BS.loc[[1,12,21,23,34,40,48,49],:]
    data2=pd.concat([data2,BS],axis=1)
data2=data2.reset_index()

In [14]:
url_list_3=[url_list[4],url_list[5],url_list[6]]
columns_a_3=[columns_a[4],columns_a[5],columns_a[6]]

data3 = pd.DataFrame()

for i,j in zip(url_list_3, columns_a_3):
    
    report=urlopen(i)
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")
    
    # 재무상태표
    p1 = parser.make2d(table[1],) 
    BS = pd.DataFrame(p1[1:],columns=j)
    BS=BS.loc[[1,10,21,23,34,39,47,48],:]
    data3=pd.concat([data3,BS],axis=1)
data3=data3.reset_index()

In [15]:
url_list_4=[url_list[8],url_list[9],url_list[10]]
columns_a_4=[columns_a[8],columns_a[9],columns_a[10]]

data4 = pd.DataFrame()

for i,j in zip(url_list_4, columns_a_4):
    
    report=urlopen(i)
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")
    
    # 재무상태표
    p1 = parser.make2d(table[1],) 
    BS = pd.DataFrame(p1[1:],columns=j)
    BS=BS.loc[[1,10,21,22,34,39,47,48],:]
    data4=pd.concat([data4,BS],axis=1)
data4=data4.reset_index()

In [16]:
report=urlopen(url_list[7])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")

# 재무상태표
p1 = parser.make2d(table[1],) 
BS = pd.DataFrame(p1[1:],columns=columns_a[7])
BS=BS.loc[[1,11,22,24,35,40,48,49],:]
BS=BS.reset_index()

In [17]:
BS_final=pd.concat([data1,data2,data3,data4,BS], axis=1)

In [18]:
BS_final = BS_final.loc[:,~BS_final.columns.duplicated()]

In [19]:
BS_final=BS_final[['구분','제 49 기 1분기말','제 49 기 반기말','제 49 기 3분기말','제 49 기말','제 50 기 1분기말','제 50 기 반기말',
                  '제 50 기 3분기말', '제 50 기말', '제 51 기 1분기말', '제 51 기 반기말', '제 51 기 3분기말']]

---------------------

#### 손익계산서

In [20]:
# 크롤링을 할때 재무상태표는 순서가 일정하지만 손익계산서와 현금흐름표는 일정하지 않아서 수동으로 하나하나 찾아줘야함.

In [21]:
data5=pd.DataFrame()

for a in [0,1,2,4,8,9,10]:
    report=urlopen(url_list[a])
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")


    # 손익계산서
    p2 = parser.make2d(table[3]) 
    IS = pd.DataFrame(p2[2:])
    IS.columns=columns_b[a]
    data5=pd.concat([data5,IS],axis=1)

In [22]:
data6=pd.DataFrame()

for a in [5,6]:
    report=urlopen(url_list[a])
    r=report.read()
    xmlsoup=BeautifulSoup(r,'html.parser')
    body=xmlsoup.find("body")
    table=body.find_all("table")


    # 손익계산서
    p2 = parser.make2d(table[4]) 
    IS = pd.DataFrame(p2[2:])
    IS.columns=columns_b[a]
    data6=pd.concat([data6,IS],axis=1)

In [23]:
report=urlopen(url_list[3])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 손익계산서
p2_3 = parser.make2d(table[3]) 
IS_3 = pd.DataFrame(p2_3[1:])
IS_3.columns=columns_b[3]

In [24]:
report=urlopen(url_list[7])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 손익계산서
p2_7 = parser.make2d(table[4]) 
IS_7 = pd.DataFrame(p2_7[1:])
IS_7.columns=columns_b[7]

In [25]:
IS_final=pd.concat([data5,data6,IS_3,IS_7],axis=1)

IS_final = IS_final.loc[:,~IS_final.columns.duplicated()]

In [26]:
IS_final_1=IS_final[['구분','제 49 기 1분기3개월','제 49 기 반기3개월','제 49 기 3분기3개월','제 49 기',
                     '제 50 기 1분기3개월','제 50 기 반기3개월','제 50 기 3분기3개월',
                     '제 51 기 1분기3개월','제 51 기 반기3개월','제 51 기 3분기3개월']]
IS_final_2=IS_final[['구분','제 49 기 1분기누적','제 49 기 반기누적','제 49 기 3분기누적', '제 49 기',
                     '제 50 기 1분기누적','제 50 기 반기누적','제 50 기 3분기누적','제 50 기',
                     '제 51 기 1분기누적','제 51 기 반기누적','제 51 기 3분기누적']]

In [27]:
IS_final_1=IS_final_1.loc[[0,2,4,9,12,14,15],:]

In [28]:
IS_final_2=IS_final_2.loc[[0,2,4,9,12,14,15],:]

------------------

#### 현금흐름표

In [29]:
report=urlopen(url_list[0])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3 = parser.make2d(table[-3])
CF0 = pd.DataFrame(p3[1:])
CF0.columns=columns_c[0]
CF0=CF0.loc[[0,9,24],:]
CF0=CF0.reset_index()

In [30]:
report=urlopen(url_list[1])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3 = parser.make2d(table[-3])
CF1 = pd.DataFrame(p3[1:])
CF1.columns=columns_c[1]
CF1=CF1.loc[[0,9,23],:]
CF1=CF1.reset_index()

In [31]:
report=urlopen(url_list[2])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3 = parser.make2d(table[-3])
CF2 = pd.DataFrame(p3[1:])
CF2.columns=columns_c[2]
CF2=CF2.loc[[0,9,23],:]
CF2=CF2.reset_index()

In [32]:
report=urlopen(url_list[3])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3 = parser.make2d(table[-3])
CF3 = pd.DataFrame(p3[1:])
CF3.columns=columns_c[3]
CF3=CF3.loc[[0,9,23],:]
CF3=CF3.reset_index()

In [33]:
report=urlopen(url_list[4])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3 = parser.make2d(table[-3])
CF4 = pd.DataFrame(p3[1:])
CF4.columns=columns_c[4]
CF4=CF4.loc[[0,9,23],:]
CF4=CF4.reset_index()

In [34]:
report=urlopen(url_list[5])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3 = parser.make2d(table[-4])
CF5 = pd.DataFrame(p3[1:])
CF5.columns=columns_c[5]
CF5=CF5.loc[[0,9,25],:]
CF5=CF5.reset_index()

In [35]:
report=urlopen(url_list[6])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3 = parser.make2d(table[-4])
CF6 = pd.DataFrame(p3[1:])
CF6.columns=columns_c[6]
CF6=CF6.loc[[0,9,25],:]
CF6=CF6.reset_index()

In [36]:
report=urlopen(url_list[7])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3_7 = parser.make2d(table[-5])
CF7 = pd.DataFrame(p3_7[1:])
CF7.columns=columns_c[7]
CF7=CF7.loc[[0,9,26],:]
CF7=CF7.reset_index()

In [37]:
report=urlopen(url_list[8])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3 = parser.make2d(table[-4])
CF8 = pd.DataFrame(p3[1:])
CF8.columns=columns_c[8]
CF8=CF8.loc[[0,9,22],:]
CF8=CF8.reset_index()

In [38]:
report=urlopen(url_list[9])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3 = parser.make2d(table[-4])
CF9 = pd.DataFrame(p3[1:])
CF9.columns=columns_c[9]
CF9=CF9.loc[[0,9,23],:]
CF9=CF9.reset_index()

In [39]:
report=urlopen(url_list[10])
r=report.read()
xmlsoup=BeautifulSoup(r,'html.parser')
body=xmlsoup.find("body")
table=body.find_all("table")


# 현금흐름표
p3 = parser.make2d(table[-4])
CF10 = pd.DataFrame(p3[1:])
CF10.columns=columns_c[10]
CF10=CF10.loc[[0,9,25],:]
CF10=CF10.reset_index()

In [41]:
CF_final=pd.concat([CF0,CF1,CF2,CF3,CF4,CF5,CF6,CF7,CF8,CF9,CF10],axis=1)

CF_final=CF_final[['구분','제 49 기 1분기',  '제 49 기 반기', '제 49 기 3분기','제 49 기','제 50 기 1분기', 
                   '제 50 기 반기', '제 50 기 3분기', '제 50 기','제 51 기 1분기',  '제 51 기 반기',  '제 51 기 3분기']]

CF_final = CF_final.loc[:,~CF_final.columns.duplicated()]

In [42]:
BS_final.head()

,구분,제 49 기 1분기말,제 49 기 반기말,제 49 기 3분기말,제 49 기말,제 50 기 1분기말,제 50 기 반기말,제 50 기 3분기말,제 50 기말,제 51 기 1분기말,제 51 기 반기말,제 51 기 3분기말
0,유동자산,"60,869,192","59,385,664","65,782,329","70,155,189","72,370,085","68,070,224","81,904,347","80,039,455","82,311,689","72,045,329","76,848,738"
1,비유동자산,"112,184,637","119,014,453","123,027,161","128,086,171","132,836,079","135,899,888","135,895,097","138,981,902","138,335,268","140,705,389","140,398,633"
2,자산총계,"173,053,829","178,400,117","188,809,490","198,241,360","205,206,164","203,970,112","217,799,444",,,,
3,유동부채,"33,947,859","36,363,120","42,071,538","44,495,084","47,168,741","40,203,756","46,368,714","43,145,053","44,319,180","34,868,869","37,245,240"
4,비유동부채,"2,527,689","2,384,321","2,454,577","2,176,501","1,842,051","1,926,719","1,771,916","43,516","44,339","45,139","46,939"


In [43]:
IS_final_1.head()

,구분,제 49 기 1분기3개월,제 49 기 반기3개월,제 49 기 3분기3개월,제 49 기,제 50 기 1분기3개월,제 50 기 반기3개월,제 50 기 3분기3개월,제 51 기 1분기3개월,제 51 기 반기3개월,제 51 기 3분기3개월
0,수익(매출액),"34,578,524","42,130,809","43,074,291","161,915,007","42,606,978","41,314,723","47,645,339","37,038,395","38,149,674","42,204,911"
2,매출총이익,"11,636,988","14,389,773","16,395,814","60,515,350","17,410,718","16,708,550","20,067,142","10,586,172","9,458,953","9,468,922"
4,영업이익(손실),"5,675,571","7,927,556","9,994,481","34,857,091","11,200,897","11,049,605","13,912,717","3,896,923","2,797,158","2,665,967"
9,법인세비용차감전순이익(손실),"6,309,744","7,927,705","10,035,363","36,533,552","11,568,977","11,094,590","13,887,163","4,154,401","4,304,753","5,786,457"
12,당기순이익(손실),"4,873,767","6,143,724","7,777,375","28,800,837","8,452,458","8,077,985","10,211,548","3,088,628","4,063,169","4,585,890"


In [44]:
IS_final_2.head()

,구분,제 49 기 1분기누적,제 49 기 반기누적,제 49 기 3분기누적,제 49 기,제 50 기 1분기누적,제 50 기 반기누적,제 50 기 3분기누적,제 50 기,제 51 기 1분기누적,제 51 기 반기누적,제 51 기 3분기누적
0,수익(매출액),"34,578,524","76,709,333","119,783,624","161,915,007","42,606,978","83,921,701","131,567,040","170,381,870","37,038,395","75,188,069","117,392,980"
2,매출총이익,"11,636,988","26,026,761","42,422,575","60,515,350","17,410,718","34,119,268","54,186,410","68,715,364","10,586,172","20,045,125","29,514,047"
4,영업이익(손실),"5,675,571","13,603,127","23,597,608","34,857,091","11,200,897","22,250,502","36,163,219","43,699,451","3,896,923","6,694,081","9,360,048"
9,법인세비용차감전순이익(손실),"6,309,744","14,237,449","24,272,812","36,533,552","11,568,977","22,663,567","36,550,730","44,398,855","4,154,401","8,459,154","14,245,611"
12,당기순이익(손실),"4,873,767","11,017,491","18,794,866","28,800,837","8,452,458","16,530,443","26,741,991","32,815,127","3,088,628","7,151,797","11,737,687"


In [45]:
CF_final.head()

,구분,제 49 기 1분기,제 49 기 반기,제 49 기 3분기,제 49 기,제 50 기 1분기,제 50 기 반기,제 50 기 3분기,제 50 기,제 51 기 1분기,제 51 기 반기,제 51 기 3분기
0,영업활동 현금흐름,"7,087,039","11,002,113","23,660,843","38,906,190","13,056,587","18,073,882","29,342,891","44,341,217","3,106,972","2,228,736","12,895,344"
1,투자활동 현금흐름,"(1,651,453)","(4,865,616)","(15,824,391)","(28,118,806)","(2,629,346)","(8,870,920)","(22,250,426)","(31,678,548)","2,866,813","4,677,057","(4,537,058)"
2,재무활동 현금흐름,"(4,331,326)","(7,297,590)","(8,658,869)","(11,801,987)","(5,421,419)","(7,984,782)","(5,936,201)","(2,716)","(3,008,810)","(5,904,161)","(8,830,958)"


In [46]:
BS_final.to_csv('data/BS.csv')
IS_final_1.to_csv('data/IS_1.csv')
IS_final_2.to_csv('data/IS_2.csv')
CF_final.to_csv('data/CF.csv')